# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [36]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

In [5]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -ls 
!hdfs dfs -mkdir {HDFS_DIR}

Found 1 items
drwxr-xr-x   - root supergroup          0 2020-01-25 16:02 demo2


# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ 

__What "programming paradigm" is Hadoop MapReduce based on?__
Functional programming-declarative programming paradigm, which means programming is done with expressions

__What are the main ideas of this programming paradigm?__
- In functional programming functions are written to avoid: sharing state, mutating data, and side-effects, i.e. the output value of a function is always the same as long as the arguments that are input to the function are the same. So calling a function f twice with the same value for an argument x will produce the same result each time.
- accept function as arguments / higher order function.

__How does MapReduce exemplify these ideas?__
Map - Applies function to each element of iterable and creates a list of the results
- Order of operations does not matter - Associative property
Reduce - also Applies function to each element in iterable 
- subject to the function being associative nad/ or commutative efficiencies can be achieved at reduce stage

> __b)__ 

__What is the Hadoop Shuffle?__
The process by which the system performs sort and transfers the map outputs to the reducers as inputs.

__When does it happen?__
between map and reduce tasks

__Why is it potentially costly?__
Because it involves writing to disk and copying data which involse I/O and network

__Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.__
map side - avoiding multiple spills to disk. Estimate the output to minimizethe number of spills
reduce side - have as much intermediate data reside in memory

> __c)__ 

__In Hadoop Streaming why do the input and output record format of a combiner script have to be the same?__
what level of combining does the framework guarantee? 
what is the relationship between the record format your mapper emits and the format your reducer expects to receive?

For the same reason why the Map output and Reduce input are the same. Combiner is executed at each buffer spill. By default combiner is optional and mapreduce framework does not guarantee to execute the combiner given to the job.

> __d)__ 

__To what extent can you control the level of parallelization of your Hadoop Streaming jobs?__
Map stage - map tasks are generally associative and can be parallelized. The optimal number of parallel tasks is largely driven by how much data we can process in memory to avoid / minimize writing to disk.
Map / Reduce tasks - it is not possible to parallelize map and reduce task as there is a hard barrier. All map tasks have to complete before reduce tasks start.
Reduce stage - degree of parallelization is dependent on the format of the output we require. I case, where in reduce task for example it is possible to sort and direct the keys to a particular reducer then we can parallelize the reducer tasks. However this is not always possible - for example in case where we need to calculate the mean of all records we would need to bring all records together.
> __e)__   

__What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop?__
the availability of low end computing resources prompted the creation of Hadoop like frameworks as these turn out to be cheaper to operate. however it requires higher number of resources which in turn requires increased ability to address faults in the process. This led to Hadoop like frameworks which enable easy parallelization of tasks and to large extent address failures of tasks in an automated manner. 

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__   

__What are counters (in the context of Hadoop Streaming)?__
Hadoop Counters provides a way to measure the progress or the number of operations that occur within map/reduce job. 
__How are they useful?__
useful channel for gathering statistics about the MapReduce job: for quality control or for application-level. They are also useful for problem diagnosis.
Hadoop Counters validate that:
-The correct number of bytes was read and written.
-The correct number of tasks was launched and successfully ran.
-The amount of CPU and memory consumed is appropriate for our job and cluster nodes.

__What kinds of counters does Hadoop provide for you?__
Built-In Counters in MapReduce
- Mapreduce Task Counters (Map input records=5)
- File system counters (FILE: Number of bytes read=60)
- Job Counters (Killed reduce tasks=1)
Custom counters in MapReduce
User-Defined Java Counters
User-Defined Streaming Counters

__How do you create your own custom counter?__
User-Defined Java Counters - by defining a java enum and then updating mapper or reducer to include conditions when the counter should increment. \
User-Defined Streaming Counters - increment counters byspecially formatted stderr stream
example from book:
sys.stderr.write('reporter:counter:Temperature, Missi0ng,1\n')

> __b)__ 

__What are composite keys?__
Composite keys are keys that have more than one attribute. An example is using:
-D stream.num.map.output.key.fields=2 : tells Hadoop to treat both the first/ key and the second / value (tab separated) fields as a composite key.
-create composite key from more columns by implementing WritableComparable interface

__How are they useful?__
Enable sorting by two different values 
Enable distribution of sorted data from mappers to reducers

__How are they related to the idea of custom partitioning?__
Example:
The First Key is responsible for data distribution across your nodes.
The Second Key is responsible for data sorting within the partition.

> __c)__ Type your answer here! 

__What is the order inversion pattern?__
can be used to control the order of reducer values in the MapReduce framework (which is useful because some computations require ordered data)
__What problem does it help solve?__
Aggregation and other types of calculation. The sorting enables implementation of logic to achieve this.
__How do we implement it?__
To provide appropriate flexibility we leverage partitioning as well as hadoop streaming command / hadoop shuffle/ sorting functionality.

# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__  

__What is the difference between a partial sort, an unordered total sort, and a total order sort?__
Partial sort - default sorting - sort input records by keys, records are only ordered relative to the other keys that happen to be processed by the same reducer.
Unordered total sort - the records with the top value didn't reliably end up in the first partition nor did the records with the lowest values necessarily end up in the last. The partition files themselves were out of order
Total order sort - When your data is sorted not only within each partition but the partitions are also in the right order

__From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total?__
concatenate the files into one to get all result in one place

__Why is this important with large datasets?__
Concatenating files is faster than sorting files to be able to concatenate them

> __b)__  

__Which phase of a MapReduce job is leveraged to implement Total Order Sort?__
mapper
__Which default behaviors must be changed. Why must they be changed?__
sort of partition keys. Because Haddop sorts by Hask Keys and these are not sorted in a human readable way

> __c)__  

__Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.__ 
Custom sorting
Partitioning

> __d)__   

__Explain why we need to use an inverse hash code function.__
to enable sort in the human readable way, as Hadoop performs sort by hask key rather than the key itself

> __e)__  

__Where does this function need to be located so that a Total Order Sort can be performed?__
defined in the mapper and used in the partition function

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [6]:
!pwd

/media/notebooks/Assignments/HW2


In [9]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [10]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l /media/notebooks/Assignments/HW2/{ENRON}

100 /media/notebooks/Assignments/HW2/data/enronemail_1h.txt


In [11]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [12]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

In [13]:
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup     204559 2020-01-26 11:22 /user/root/HW2/enron.txt


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output from your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop  MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected, then run the provided Hadoop Streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Explain why or why not. (`HINT:` This question demands an understanding of the sequence of the phases of MapReduce.)


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Rewrite your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ \

assistance	1	8 \
assistance	0	2 

> __c)__ 

__sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count__
the results will be partitioned before they are sorted. Therefore even though we can apply sort by count it will be applied after partitioning and therefore it would not provide the desired result.

> __d)__ 

__how you would go about printing the top 10 words in each class given this sorted output__
1. get top 10 from each file filtered by class 
2. concatenate 
3. Sort by class and count - reverse
4. Display top 10 aka head
while this is not too onerous for two files this would be more complicated with increased number of files

In [14]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [15]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [32]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	1	0
two	0	1


In [37]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

rm: `/user/root/HW2/eda-output': No such file or directory


In [38]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

rm: `/user/root/HW2/eda-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7889522164996968704.jar tmpDir=null
20/01/26 15:05:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 15:05:54 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 15:05:55 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/26 15:05:55 INFO mapreduce.JobSubmitter: number of splits:2
20/01/26 15:05:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0027
20/01/26 15:05:55 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0027
20/01/26 15:05:55 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0027/
20/01/26 15:05:55 INFO mapreduce.Job: Running job: job_1579903014542_0027
20/01/26 15:06:02 INFO mapreduce.Job: Job job_1579903014542_0027 running in uber mode : false
20

In [39]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [40]:
# part b - write your grep command here
!grep 'assistance' EnronEDA/results.txt

assistance	1	8
assistance	0	2


In [46]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [49]:
# part d - write your Hadoop streaming job here
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k3,3nr" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

Deleted /user/root/HW2/eda-sort-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8844326108999041629.jar tmpDir=null
20/01/26 15:36:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 15:36:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 15:36:05 INFO mapred.FileInputFormat: Total input paths to process : 2
20/01/26 15:36:05 INFO mapreduce.JobSubmitter: number of splits:2
20/01/26 15:36:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0031
20/01/26 15:36:06 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0031
20/01/26 15:36:06 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0031/
20/01/26 15:36:06 INFO mapreduce.Job: Running job: job_1579903014542_0031
20/01/26 15:36:13 INFO mapreduce.Job: Job job_1579903014542_0031 running in uber mode : false
20/01/26 15:36:13 INFO

In [50]:
# part d - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

and	1	392	
a	1	347	
you	1	345	
of	1	336	
for	1	204	
it	1	152	
that	1	145	
on	1	136	
is	1	135	
will	1	102	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
your	1	357	
in	1	236	
com	1	153	
this	1	143	
i	1	140	
or	1	117	
we	1	116	
with	1	116	
cat: Unable to write to output stream.


In [ ]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

In [44]:
# part e - write your Hadoop streaming job here
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

Deleted /user/root/HW2/eda-sort-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob3006420551311855225.jar tmpDir=null
20/01/26 15:31:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 15:31:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 15:31:10 INFO mapred.FileInputFormat: Total input paths to process : 2
20/01/26 15:31:10 INFO mapreduce.JobSubmitter: number of splits:2
20/01/26 15:31:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0029
20/01/26 15:31:11 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0029
20/01/26 15:31:11 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0029/
20/01/26 15:31:11 INFO mapreduce.Job: Running job: job_1579903014542_0029
20/01/26 15:31:19 INFO mapreduce.Job: Job job_1579903014542_0029 running in uber mode : false
20/01/26 15:31:19 INFO

In [45]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__   

__how many records are emitted by the mappers__
Map output records=31490
__how many records are received be the reducers__
Reduce input records=31490
__In the context of word counting what does this number represent practically?__
in the context of our use case this number represents total number of words in our source data, in our case title and body.

> __c)__ 

__how many records are emitted by the mappers__
Map output records=31490
__how many records are received be the reducers__
Reduce input records=13096
__Compare your results here to what you saw in part a. Explain.__
A reduce task has been applied on the output of each mapper decreasing the size of the data that had to be moved towards the reducer. In this specific case we can see a significant reduction.

4a
Reduce output records=10130
5b
Reduce output records=10130

> __d)__ 

__Describe a scenario where using a combiner would NOT improve the efficiency of the shuffle stage. Explain.__
- Combiner can be used just in case the reduce function is both commutative and associative. It's because values are combined locally before shuffle in arbitrary order.
- Combiner is unlikely to increase efficiency in cases where map is generating low number of outputs against the same key.

__how does increasing the number of mappers affect the usefulness of a combiner?__

Combiner is advised in case your map is generating very large number of output against the same key.

with increased number of mappers there is a higher likelihood that the records, in particular where the records do not have high counts, will be distributed across multiple mappers and therefore lowering the effectiveness of the combiner.
However, in cases where the counts would be high / repetition of the same word is high the combiners can provide significant performance benefits.


In [51]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [52]:
# part b - write your Hadoop streaming job here
!hdfs dfs -rm -r {HDFS_DIR}/eda-output
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

rm: `/user/root/HW2/eda-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7714014552601817502.jar tmpDir=null
20/01/26 16:26:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 16:26:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/26 16:26:07 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/26 16:26:07 INFO mapreduce.JobSubmitter: number of splits:2
20/01/26 16:26:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0032
20/01/26 16:26:08 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0032
20/01/26 16:26:08 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0032/
20/01/26 16:26:08 INFO mapreduce.Job: Running job: job_1579903014542_0032
20/01/26 16:26:15 INFO mapreduce.Job: Job job_1579903014542_0032 running in uber mode : false
20

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ 

__In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$?__
- mathematical notation and \
1.$\hat{P}(c)$ \
2. $\hat{P}(t_{k} | c)$  

- verbal explanation \
  1. prior probability of a document occurring in a class
  2. we need conditional probability of term 'word' occurring in a document of a particular class  \


> __b)__ 

explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data.
- $\hat{P}(c)$ - overall probability of a case being 'chinese'
- $\hat{P}(t_{k} | c)$ probability of a word occurring in the class 'chinese'

> __c)__ Show your calculations here using markdown & $\LaTeX$ or embed them below!

__Priors__ \
$P(c) = \frac{3}{4}$ \
$P(c') = \frac{1}{4}$ \
__Conditional__ 
- about China \
$\hat{P}(Chinese| c) = \frac{5}{8} $ \
$\hat{P}(Tokyo | c) = \hat{P}(Japan | c) = \frac{0}{8}  $ 
- Not about China \
$\hat{P}(Chinese| c') = \frac{1}{3} $ \
$\hat{P}(Tokyo | c') = \hat{P}(Japan | c') = \frac{1}{3}  $  

> __d)__ Show your calculations here using markdown & $\LaTeX$ or embed them below!

$\hat{P}(c | d5)  \;\alpha  \;\frac{3}{4} \;\times \;(\frac{5}{8})^{3} \;\times \frac{0}{8} \;\times \frac{0}{8} \approx 0$


$\hat{P}(c' | d5)  \;\alpha  \;\frac{1}{4} \;\times \;(\frac{1}{3})^{3} \;\times \frac{1}{3} \;\times \frac{1}{3} \approx 0.001028$


> __e)__ 

__How does smoothing affect our inference?__
makes our classifications less sensitve to rare words


In [ ]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ 

__What are the 'keys' and 'values' in this file?__
- keys - cities & ClassPriors
- values - count of city in the doc class 'not about china' , count of city in the doc class 'about china' , conditional probability of not about china, conditional probability of about china

__Which record means something slightly different than the rest?__ 
ClassPriors - prior probability of a doc being about or not about China

__The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they?__
- count of city in the doc class 'not about china'
- count of city in the doc class 'about china'

> __b)__ 

__Why do we choose to work with log probabilities?__
to avoid floating point underflow - a condition in a computer program where the result of a calculation is a number of smaller absolute value than the computer can actually represent in memory on its central processing unit (CPU)

__If we had an unsmoothed model, what potential error could arise from this transformation?__
the conditional probabillity is 0. A log of zero does not exist.

> __c)__

__What should we do at inference time when we need a class conditional probability for this word?__
I choose to set the probability to 1. log of one equals 0. As i know i will be adding all conditinal probabilities this should enable me get pass the limitation and not have adverse effect on the accuracy of the prediction.

> __d)__

__Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not.__
Not really. We are using an object but we are not changing the actual state or modifying it in any way.

__From a scability perspective when would this kind of memory use be justified? when would it be unwise?__
the difficulty would be if the model is too large to fit into the memory as this would be a scalability limitation. Other consideration is that we need to ship all the information to the mapper nodes across the network, however this  consideration is independent on how this data is then accessed within the mapper itself.

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__

__In this hypothetical scenario how would our algorithm design change?__ 
If we are unable to process everything within memory then we would have to add one additional step to aggregate the conditional probabilities. We would want to partition the mapper data and aggregate the probability in the reducer.

__What could the mappers still do? What key-value structure would they emit?__ 
The mapper could emit one line summary for the entire partition. As these are log probabillities the reducer only needs to sum all of the individual partitions. The mapper woudl not be able to determine the predicted class though.

__What would the reducers have to do as a last step?__
aggregate the probability in the reducer and predict class.

Run these cells to create the example corpus and model.

In [53]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Writing NaiveBayes/chineseTrain.txt


In [54]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Writing NaiveBayes/chineseTest.txt


In [55]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Writing NBmodel.txt


In [89]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NBmodel.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/chineseTrain.txt': File exists
copyFromLocal: `/user/root/HW2/chineseTest.txt': File exists
copyFromLocal: `/user/root/HW2/NBmodel.txt': File exists


Your work for `part e` starts here:

In [92]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py
!chmod a+x NBmodel.txt

In [381]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [375]:
MODEL = {}
for record in open('NBmodel.txt', 'r').readlines():
    word, payload = record.split('\t')
    # extract conditional probabilities
    ham_cProb, spam_cProb = payload.split(',')[2:]
    # save their logs as a tuple in our model dictionary
    take_log = lambda x: np.log(x) if x != 0 else float("-inf")
    MODEL[word] = (take_log(float(ham_cProb)),
                   take_log(float(spam_cProb)))
print(MODEL)

{'beijing': (-2.197224577337219, -1.9459101490563133), 'chinese': (-1.504077396777274, -0.8472978603882036), 'tokyo': (-1.504077396777274, -2.6390573296148587), 'shanghai': (-2.197224577337219, -1.9459101490563133), 'ClassPriors': (-1.3862943611198906, -0.2876820724517809), 'japan': (-1.504077396777274, -2.6390573296148587), 'macao': (-2.197224577337219, -1.9459101490563133)}


In [81]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

Deleted /user/root/HW2/chinese-output


In [378]:
# part e - write your Hadooop streaming job here
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output
!hadoop jar {JAR_FILE} \
  -files NBmodel.txt,NaiveBayes/classify_mapper.py \
  -mapper classify_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

Deleted /user/root/HW2/chinese-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2071627984914102179.jar tmpDir=null
20/01/30 22:03:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/30 22:03:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/30 22:03:38 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/30 22:03:38 INFO mapreduce.JobSubmitter: number of splits:2
20/01/30 22:03:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0098
20/01/30 22:03:38 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0098
20/01/30 22:03:38 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0098/
20/01/30 22:03:38 INFO mapreduce.Job: Running job: job_1579903014542_0098
20/01/30 22:03:46 INFO mapreduce.Job: Job job_1579903014542_0098 running in uber mode : false
20/01/30 22:03:46 INFO 

In [379]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [380]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). __IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. 

    - [`HINT:` You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)] 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 



__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __ c)__ count of unique words

> __ d)__ 
I am not sure how i would achieve the option 1, i would think that I have to pass over the entire dataset, sort and compare records and count. Whereas for option 2 - with postprocessing, we will have the count of unique words at the end of the reducer job and only for the individual partition whereas we need it when we are calculating each record from the start of the reducer job.

> __ e)__ 
I expect postprocessing to be faster, as the number of records to pass over should be smaller. however the postprocessing means that I have to carry out the calculation of the conditional probability as part of the processing. In reality I would probably two approaches. one as described. Another implementing two mapreduce jobs. map + aggregate reduce >> map with unique + calculate reduce.

In [442]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!chmod a+x NaiveBayes/train_reducer_smooth.py
!chmod a+x NaiveBayes/smooth_postprocess.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount  
OUTPUT:
    word \t {spamCount, hamCount, ham_cProb, spam_cProb}


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [414]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2 | column -t

A  !ClassWords  3  8
B  !ClassWords  3  8
B  ClassPriors  1  3
A  beijing      0  1
B  chinese      0  1
B  chinese      0  1
B  chinese      0  1
B  chinese      0  1
B  chinese      0  1
B  chinese      1  0
A  japan        1  0
B  macao        0  1
B  shanghai     0  1
A  tokyo        1  0


In [480]:
# part b - write a unit test for your reducer here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2| \
NaiveBayes/train_reducer.py | column -t

ClassPriors  1,3,0.25,0.75
beijing      0,1,1,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,1
macao        0,1,1,0.125
shanghai     0,1,1,0.125
tokyo        1,0,0.3333333333333333,1


In [327]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2| \
NaiveBayes/train_reducer.py | sort | smooth_postprocess.py | column -t

ClassPriors  1,3,0.25,0.75
beijing      0,1,0.0,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,0.0
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
tokyo        1,0,0.3333333333333333,0.0


In [228]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/chinese-unsmoothed

Deleted /user/root/HW2/chinese-unsmoothed


In [481]:
# part b - write your hadoop streaming job
!hdfs dfs -rm -r {HDFS_DIR}/chinese-unsmoothed
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2 " \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/chinese-unsmoothed \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2

Deleted /user/root/HW2/chinese-unsmoothed
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob3886275266922781668.jar tmpDir=null
20/01/31 18:52:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 18:52:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 18:52:57 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/31 18:52:58 INFO mapreduce.JobSubmitter: number of splits:2
20/01/31 18:52:58 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0127
20/01/31 18:52:58 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0127
20/01/31 18:52:58 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0127/
20/01/31 18:52:58 INFO mapreduce.Job: Running job: job_1579903014542_0127
20/01/31 18:53:05 INFO mapreduce.Job: Job job_1579903014542_0127 running in uber mode : false
20/01/31 18:53:05 I

In [329]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/chinese-unsmoothed/part-000* > NaiveBayes/UnsmoothedResults.txt

In [330]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/UnsmoothedResults.txt | column -t

ClassPriors  1,3,0.25,0.75
chinese      1,5,0.3333333333333333,0.625
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
beijing      0,1,0.0,0.125
japan        1,0,0.3333333333333333,0.0
tokyo        1,0,0.3333333333333333,0.0


In [288]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [482]:
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2 | column -t

A  !ClassWords  3  8
A  ClassPriors  1  3
A  beijing      0  1
A  chinese      0  1
A  chinese      0  1
A  chinese      0  1
A  chinese      0  1
A  chinese      0  1
A  chinese      1  0
A  japan        1  0
A  macao        0  1
A  shanghai     0  1
A  tokyo        1  0


In [483]:
# part e - write a unit test for your NEW reducer here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2| \
NaiveBayes/train_reducer.py | column -t

ClassPriors  1,3,0.25,0.75
beijing      0,1,1,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,1
macao        0,1,1,0.125
shanghai     0,1,1,0.125
tokyo        1,0,0.3333333333333333,1


In [484]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2,2| \
NaiveBayes/train_reducer.py | column -t

ClassPriors  1,3,0.25,0.75
beijing      0,1,1,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,1
macao        0,1,1,0.125
shanghai     0,1,1,0.125
tokyo        1,0,0.3333333333333333,1


In [447]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/chinese-smoothed

Deleted /user/root/HW2/chinese-smoothed


In [463]:
# part e - write your hadoop streaming job
!hdfs dfs -rm -r {HDFS_DIR}/chinese-smoothed
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2 " \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/chinese-smoothed \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2

Deleted /user/root/HW2/chinese-smoothed
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8274564141827684881.jar tmpDir=null
20/01/31 00:41:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 00:41:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 00:41:41 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/31 00:41:41 INFO mapreduce.JobSubmitter: number of splits:2
20/01/31 00:41:41 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0118
20/01/31 00:41:42 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0118
20/01/31 00:41:42 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0118/
20/01/31 00:41:42 INFO mapreduce.Job: Running job: job_1579903014542_0118
20/01/31 00:41:49 INFO mapreduce.Job: Job job_1579903014542_0118 running in uber mode : false
20/01/31 00:41:49 INF

In [466]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/chinese-smoothed/part-000* | sort | NaiveBayes/smooth_postprocess.py > NaiveBayes/SmoothedResults.txt

!cat NaiveBayes/SmoothedResults.txt | column -t

ClassPriors  1,3,0.25,0.75
beijing      0,1,0.1111111111111111,0.14285714285714285
chinese      1,5,0.2222222222222222,0.42857142857142855
japan        1,0,0.2222222222222222,0.07142857142857142
macao        0,1,0.1111111111111111,0.14285714285714285
shanghai     0,1,0.1111111111111111,0.14285714285714285
tokyo        1,0,0.2222222222222222,0.07142857142857142


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. [`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]



### Q9 Student Answers:
> __d)__ 

__Compare the performance of your two models.__
smoothing definitely improves the model performance

__What do you notice about the unsmoothed model's predictions?__
it would seem that the smoothed results for ham / spam are a lot closer to one another than the unsmoothed results

__Can you guess why this is happening?__
becuase our classifications are less sensitve to rare words

__Which evaluation measure do you think is most relevant in our use case?__
precision and recall

__Test/Train split__

In [289]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [398]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2 " \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

Deleted /user/root/HW2/enron-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob1738499756421908179.jar tmpDir=null
20/01/30 22:43:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/30 22:43:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/30 22:43:42 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/30 22:43:42 INFO mapreduce.JobSubmitter: number of splits:2
20/01/30 22:43:42 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0111
20/01/30 22:43:42 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0111
20/01/30 22:43:42 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0111/
20/01/30 22:43:42 INFO mapreduce.Job: Running job: job_1579903014542_0111
20/01/30 22:43:49 INFO mapreduce.Job: Job job_1579903014542_0111 running in uber mode : false
20/01/30 22:43:49 INFO map

In [488]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2,4,0.0001725476662928134,0.00029682398337785694


In [489]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1,22,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [469]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k2,2 " \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model \
  -cmdenv PATH={PATH} \
  -numReduceTasks 2

# apply POST PROCESS and save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* | sort | NaiveBayes/smooth_postprocess.py> NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob5053958865615182082.jar tmpDir=null
20/01/31 00:47:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 00:47:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 00:47:15 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/31 00:47:15 INFO mapreduce.JobSubmitter: number of splits:2
20/01/31 00:47:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0120
20/01/31 00:47:15 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0120
20/01/31 00:47:15 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0120/
20/01/31 00:47:15 INFO mapreduce.Job: Running job: job_1579903014542_0120
20/01/31 00:47:23 INFO mapreduce.Job: Job job_1579903014542_0120 running in uber mode : false
20/01/31 00:47:23 INFO ma

In [470]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2,4,0.0001858045336306206,0.00027730020520215184


In [471]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1,22,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [474]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [544]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	6.134092622766479	7.532326167942548	1
d6	1	3.0081547935545476	3.6041382256608454	1
d7	0	3.819085009771877	6.93634273583425	1
d8	0	1.6218604324346575	4.990432586777937	1
d5	1	6.134092622766479	7.532326167942548	 True
d6	1	3.0081547935545476	3.6041382256608454	 True
d7	0	3.819085009771877	6.93634273583425	 False
d8	0	1.6218604324346575	4.990432586777937	 False
# Documents: 	4.0
True Positives:	2.0
True Negatives:	0.0
False Positives:	2.0
False Negatives:	0.0
Accuracy:	0.5
Precision:	0.5
Recall:	1.0
F-Score:	0.6666666666666666


In [545]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/unsmooth-results

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/Unsmoothed/NBmodel.txt,NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/unsmooth-results \
  -cmdenv PATH={PATH} \

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/unsmooth-results/part-000* > NaiveBayes/Unsmoothed/results.txt
!cat NaiveBayes/Unsmoothed/results.txt | column -t

Deleted /user/root/HW2/unsmooth-results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob1169574045150028489.jar tmpDir=null
20/01/31 21:37:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 21:37:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 21:37:17 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/31 21:37:18 INFO mapreduce.JobSubmitter: number of splits:2
20/01/31 21:37:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0147
20/01/31 21:37:18 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0147
20/01/31 21:37:18 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0147/
20/01/31 21:37:18 INFO mapreduce.Job: Running job: job_1579903014542_0147
20/01/31 21:37:25 INFO mapreduce.Job: Job job_1579903014542_0147 running in uber mode : false
20/01/31 21:37:25 INF

In [546]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-results

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/Smoothed/NBmodel.txt,NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/smooth-results \
  -cmdenv PATH={PATH} \

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/smooth-results/part-000* > NaiveBayes/Smoothed/results.txt
!cat NaiveBayes/Smoothed/results.txt | column -t

Deleted /user/root/HW2/smooth-results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4751150189798274221.jar tmpDir=null
20/01/31 21:38:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 21:38:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 21:38:04 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/31 21:38:04 INFO mapreduce.JobSubmitter: number of splits:2
20/01/31 21:38:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0148
20/01/31 21:38:04 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0148
20/01/31 21:38:04 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0148/
20/01/31 21:38:04 INFO mapreduce.Job: Running job: job_1579903014542_0148
20/01/31 21:38:12 INFO mapreduce.Job: Job job_1579903014542_0148 running in uber mode : false
20/01/31 21:38:12 INFO 

In [547]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
# Documents: 	20.0
True Positives:	2.0
True Negatives:	0.0
False Positives:	9.0
False Negatives:	9.0
Accuracy:	0.1
Precision:	0.18181818181818182
Recall:	0.18181818181818182
F-Score:	0.18181818181818182
=========== SMOOTHED MODEL ============
# Documents: 	20.0
True Positives:	11.0
True Negatives:	6.0
False Positives:	3.0
False Negatives:	0.0
Accuracy:	0.85
Precision:	0.7857142857142857
Recall:	1.0
F-Score:	0.88


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

# Question 10: Custom Partitioning and Secondary Sort

Now that we have our model, we can analyse the results and think about future improvements.

### Q10 Tasks:

* __a) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __b) short response:__ What do you notice about the 'top words' we printed in `a`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q10 Student Answers:
> __a)__ 

by partitioning by key3 ham/spam and sorting by maxClassP

> __b)__ 

__What do you notice about the 'top words' we printed in `a`?__ 
a lot of short words prepositions, a, the, etc

__How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'.__
I expect the prominence of these words to increase as smoothing decreses the impact of rare words. And the divide between the ham/spam respectively to increase.

__How would they affect the probabilities of words that occur much more in one class than another?__ it would move probabilities towards more uninformed mean. If large values are used than the influence of observed counts will be lower because estimated probabilities for the same number of observations will be lower. 

__In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model.__
A direct consequence is that the variance of the model (estimates) is lower, possibly with larger bias. Y


In [518]:
# Copy the model 
PWD = !pwd
PWD = PWD[0]
TEST_TXT = PWD + "/NaiveBayes/Smoothed/NBmodel.txt"

!hdfs dfs -put {TEST_TXT} {HDFS_DIR}/NBmodelSmooth.txt

put: `/user/root/HW2/NBmodelSmooth.txt': File exists


In [530]:
# part a - write your Hadoop job here (sort smoothed model on P(word|class))

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/sorted-model

# hadoop job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k3,3" \
  -D mapreduce.partition.keycomparator.options="-k4,4 -nr" \
  -files NaiveBayes/model_sort_mapper.py \
  -mapper model_sort_mapper.py \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/NBmodelSmooth.txt \
  -output {HDFS_DIR}/sorted-model \
  -cmdenv PATH={PATH} \

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/sorted-model/part-000* > NaiveBayes/Smoothed/sortedResults.txt
!head -n 20 NaiveBayes/Smoothed/sortedResults.txt | column -t

Deleted /user/root/HW2/sorted-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob720256677015459374.jar tmpDir=null
20/01/31 20:22:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 20:22:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/01/31 20:22:49 INFO mapred.FileInputFormat: Total input paths to process : 1
20/01/31 20:22:49 INFO mapreduce.JobSubmitter: number of splits:2
20/01/31 20:22:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1579903014542_0134
20/01/31 20:22:49 INFO impl.YarnClientImpl: Submitted application application_1579903014542_0134
20/01/31 20:22:49 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1579903014542_0134/
20/01/31 20:22:49 INFO mapreduce.Job: Running job: job_1579903014542_0134
20/01/31 20:22:57 INFO mapreduce.Job: Job job_1579903014542_0134 running in uber mode : false
20/01/31 20:22:57 INFO map

In [529]:
# part b - print top words in each class
print('=========== PART-00000 ============')
!awk '{ if ($3 == "ham") { print } }' NaiveBayes/Smoothed/sortedResults.txt | sort -k4 -nr | head -n 10 
print('=========== PART-00001 ============')
!awk '{ if ($3 == "spam") { print } }' NaiveBayes/Smoothed/sortedResults.txt | sort -k4 -nr | head -n 10 

=========== PART-00000 ============
ect	378,0,0.023473306082001735,5.546004104043037e-05	ham	0.023473306082001735	
and	258,277,0.01604112473677691,0.015417891409239643	ham	0.01604112473677691	
hou	203,0,0.0126347082868822,5.546004104043037e-05	ham	0.0126347082868822	
in	160,157,0.009971509971509971,0.008762686484387999	ham	0.009971509971509971	
for	148,153,0.00922829183698749,0.008540846320226277	ham	0.00922829183698749	
on	122,95,0.007617985878855444,0.005324163939881316	ham	0.007617985878855444	
enron	116,0,0.007246376811594203,5.546004104043037e-05	ham	0.007246376811594203	
will	113,69,0.007060572277963582,0.003882202872830126	ham	0.007060572277963582	
i	113,106,0.007060572277963582,0.00593422439132605	ham	0.007060572277963582	
this	99,90,0.00619348445435402,0.005046863734679163	ham	0.00619348445435402	
sort: write failed: standard output: Broken pipe
sort: write error
=========== PART-00001 ============
the	453,535,0.02811841942276725,0.029726581997670677	spam	0.029726581997670677	

### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform